In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3

/content/gdrive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3


In [ ]:
ls

detr/  generate_panoptic_data.ipynb  Panoptic-DETR.ipynb


In [ ]:
#!git clone https://github.com/facebookresearch/detr.git

In [ ]:
cd detr

/content/gdrive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr


In [ ]:
ls

coco/                       Dockerfile  models/           run_with_submitit.py
coco_panoptic/              engine.py   output_2/         test_all.py
d2/                         hubconf.py  __pycache__/      tox.ini
datasets/                   LICENSE     README.md         util/
detr-r50_no-class-head.pth  main.py     requirements.txt


In [ ]:
!pip install git+https://github.com/cocodataset/panopticapi.git

  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-lp05ieor
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-lp05ieor
  Created wheel for panopticapi: filename=panopticapi-0.1-py3-none-any.whl size=8306 sha256=ab156ac3c86443a3c1b7f896eaee2630a5f8f72457a42e214a1d13c3ff5a1a76
  Stored in directory: /tmp/pip-ephem-wheel-cache-vnq5ege5/wheels/ad/89/b8/b66cce9246af3d71d65d72c85ab993fd28e7578e1b0ed197f1
Successfully built panopticapi


In [ ]:
import torch
# Get pretrained weights
checkpoint = torch.hub.load_state_dict_from_url(
            url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
            map_location='cpu',
            check_hash=True)

# Remove class weights
del checkpoint["model"]["class_embed.weight"]
del checkpoint["model"]["class_embed.bias"]

# Save
torch.save(checkpoint,
           'detr-r50_no-class-head.pth')

Downloading: "https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth" to /root/.cache/torch/hub/checkpoints/detr-r50-e632da11.pth


  0%|          | 0.00/159M [00:00<?, ?B/s]

# Training for BBOX detection (DETR Object) for 3 classes

### Note- The checkpoint.pth generated from this training needs to be saved separately and re-used as input for panoptic training.

In [ ]:
 first_class_index = 0
 num_classes = 3
 finetuned_classes = ['rebar','spall','crack']

In [ ]:
!python main.py \
--dataset_file "coco" \
--coco_path "/content/gdrive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco" \
--output_dir "/content/gdrive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/output_2" \
--resume "detr-r50_no-class-head.pth" \
--batch_size 4 \
--num_classes 3 \
--epochs 1 

Not using distributed mode
git:
  sha: 8a144f83a287f4d3fece4acdf073f387c5af387d, status: has uncommited changes, branch: main

Namespace(aux_loss=True, backbone='resnet50', batch_size=4, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path=None, coco_path='/content/gdrive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco', dataset_file='coco', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=1, eval=False, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=0.0001, lr_backbone=1e-05, lr_drop=200, mask_loss_coef=1, masks=False, nheads=8, num_classes=3, num_queries=100, num_workers=2, output_dir='/content/gdrive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/output_2', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='detr-r50_no-class-head.pth', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, weigh

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=1 --use_env main.py --coco_path coco  --coco_panoptic_path coco_panoptic  --output_dir output_2 --batch_size 6 --epochs 100 --resume output_2/checkpoint.pth 

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=1 --use_env main.py --coco_path coco  --coco_panoptic_path coco_panoptic  --output_dir output_2 --batch_size 6 --epochs 200 --resume output_2/checkpoint.pth 

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=1 --use_env main.py --coco_path coco  --coco_panoptic_path coco_panoptic  --output_dir output_2 --batch_size 6 --epochs 250 --resume output_2/checkpoint.pth 